In [1]:
# useing this variable for saving models and summaries
exp_name = "CNN_activity_Wmultitask18_64_opp"

## Importing Libraries

In [2]:
%load_ext autoreload
%autoreload 2
import tensorflow as tf
import numpy as np
from hdf5storage import savemat
import hickle 

import sys
sys.path.append('../src')

In [3]:
import models.CNN_model as CNN_model
import training
import evaluation.nn_eval_multitask as nn_eval

## Loading synthetic data

In [4]:
#loading data
data_path= '../data/opp8.hkl'
with open(data_path, 'r') as fin:
    data = hickle.load(fin)

In [5]:
train_data = data['train_data']
validation_data = data['validation_data']
test_data = data['test_data']

train_gt = data['train_gt']
validation_gt = data['validation_gt']
test_gt = data['test_gt']

In [6]:
train_data.shape, train_gt.shape

((691, 1024, 77), (691, 1024, 19))

In [7]:
test_data.shape, test_gt.shape

((119, 1024, 77), (119, 1024, 19))

## Model

In [8]:
# Model parameters
n_variables = 77
learning_rate = 0.001
batch_size = 100

convolution_levels = 3    #number of convolution to have in the network
pooling_stride = [4, 4, 4]
weights_shape = [[9, n_variables, 32],
                 [5, 32, 32],
                 [5, 32, 32]]
bias_shape = [32, 32, 32]
activation = tf.nn.relu
n_classes = 19

##positive weight for weighted cross entropy
s = train_gt.shape
resolution = np.prod(pooling_stride)
pooled_gt = np.reshape(train_gt[:,:(s[0]*s[1]/resolution)*resolution,:], (s[0]*(s[1]/resolution), resolution, s[2])).max(axis=1)
pos_weight = (1.0 - pooled_gt.mean(axis = 0)) / pooled_gt.mean(axis=0)


variables_weights = np.ones((n_classes,))
variables_weights[-1] = 18

In [9]:
pos_weight.shape

(19,)

In [10]:
pos_weight

array([  0.16329966,  34.21019108,  34.43589744,  37.12413793,
        36.10067114,  26.70927318,  27.27621483,  39.94814815,
        42.01945525,  46.65517241,  51.89952153,  48.57847534,
        51.1509434 ,  43.40160643,  44.68595041,  33.7672956 ,
        10.65015806,  36.60544218,   2.20649652])

In [11]:
# Launch the graph
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

In [12]:
summary_path = '../summary/' + exp_name 
checkpoint_path = '../model/' + exp_name + '/' + exp_name 

In [13]:
model = CNN_model.Model(n_variables = n_variables,
                        learning_rate = learning_rate,
                        n_classes = n_classes)

In [14]:
model.build_conv_net(pos_weight_values = pos_weight,
                     learning_rate = learning_rate,
                     convolution_levels = convolution_levels, 
                     pooling_stride = pooling_stride,
                     weights_shape = weights_shape,
                     bias_shape = bias_shape,
                     n_classes = n_classes,
                     activation = activation,
                     trainable_conv = True,
                     trainable_last = True,
                     variables_weights = variables_weights)

In [17]:
trainer = training.Trainer(model = model,
                    data_train = train_data,
                    ground_truth_train = train_gt,
                    data_validation = test_data,
                    ground_truth_validation = test_gt,
                    sess = sess,
                    summary_path = summary_path,
                    batch_size = batch_size,
                    global_step = 0,
                    eval_type = 'multitask')

In [18]:
trainer.train(max_iter = 20000,
                train_eval_step = 100, 
                validation_eval_step = 100,
                display_step = 100)

Train Batch Evaluation
ACTIVITY RECOGNITION
AUC = 0.079 vs. 0.076
F1 = 0.130 vs. 0.115
Accuracy = 0.017 vs. 0.056
CHANGE DETECTION
AUC = 0.241 vs. 0.291
F1 = 0.453 vs. 0.453
Precision = 0.293 vs. 0.294
Recall = 0.996 vs. 0.987
Validation
ACTIVITY RECOGNITION
AUC = 0.073 vs. 0.068
F1 = 0.106 vs. 0.099
Accuracy = 0.009 vs. 0.056
CHANGE DETECTION
AUC = 0.165 vs. 0.175
F1 = 0.312 vs. 0.313
Precision = 0.185 vs. 0.188
Recall = 1.000 vs. 0.952
Minibatch Loss= 1.200482
Train Batch Evaluation
ACTIVITY RECOGNITION
AUC = 0.557 vs. 0.077
F1 = 0.601 vs. 0.115
Accuracy = 0.445 vs. 0.056
CHANGE DETECTION
AUC = 0.871 vs. 0.323
F1 = 0.807 vs. 0.476
Precision = 0.778 vs. 0.312
Recall = 0.838 vs. 1.000
Validation
ACTIVITY RECOGNITION
AUC = 0.352 vs. 0.071
F1 = 0.441 vs. 0.101
Accuracy = 0.478 vs. 0.056
CHANGE DETECTION
AUC = 0.650 vs. 0.196
F1 = 0.622 vs. 0.315
Precision = 0.575 vs. 0.189
Recall = 0.678 vs. 0.940
Minibatch Loss= 0.447006
Train Batch Evaluation
ACTIVITY RECOGNITION
AUC = 0.679 vs. 0.083


KeyboardInterrupt: 

In [19]:
#Save the model
saver = tf.train.Saver()
saver.save(sess, checkpoint_path, global_step=trainer.global_step)

'../model/CNN_activity_Wmultitask18_64_opp/CNN_activity_Wmultitask18_64_opp-10325'

In [21]:
#load the model
saver = tf.train.Saver()
#saver.restore(sess, "../model" + "/CNN7002_Gleam" + "/CNN7002_Gleam")
saver.restore(sess, '../model/CNN_activity_multitask_64_opp/CNN_activity_multitask_64_opp-20000')


INFO:tensorflow:Restoring parameters from ../model/CNN_activity_multitask_64_opp/CNN_activity_multitask_64_opp-20000


In [20]:
#This is the model evaluation on test dataset
_= nn_eval.evaluate_model (model, sess, test_data, test_gt)

ACTIVITY RECOGNITION
AUC = 0.549 vs. 0.068
F1 = 0.605 vs. 0.097
Accuracy = 0.881 vs. 0.056
CHANGE DETECTION
AUC = 0.650 vs. 0.187
F1 = 0.651 vs. 0.315
Precision = 0.579 vs. 0.189
Recall = 0.744 vs. 0.946


In [ ]:
#shape of probability_change is (batch size, T, num_classes)
#since num_classes = 1 we squeeze that axis
probability_change = np.squeeze(probability_of_pos, axis = 2)

In [ ]:
savemat('../poc/poc'+exp_name+'.mat', {'poc':probability_change})

In [ ]:
1